In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
X = df.drop('label',axis=1)

In [5]:
y=df['label']

In [6]:
df.shape

(20800, 5)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer

In [8]:
df=df.dropna()

In [9]:
df.shape

(18285, 5)

In [10]:
messages=df.copy()

In [11]:
messages.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
messages.reset_index(inplace=True)

In [13]:
messages.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [14]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps = PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    review = re.sub('[^a-z-A-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [15]:
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu - breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [16]:
#apply bag of words
cv = CountVectorizer(max_features=5000,ngram_range=(1,3))
X = cv.fit_transform(corpus).toarray()

In [17]:
X.shape

(18285, 5000)

In [18]:
y=messages['label']

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=0)

In [20]:
cv.get_feature_names()[:20]

['abandon',
 'abc',
 'abc news',
 'abduct',
 'abe',
 'abedin',
 'abl',
 'abort',
 'abroad',
 'absolut',
 'abstain',
 'absurd',
 'abus',
 'abus new',
 'abus new york',
 'accept',
 'access',
 'access pipelin',
 'access pipelin protest',
 'accid']

In [21]:
cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 5000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [22]:
count_df = pd.DataFrame(X_train,columns=cv.get_feature_names())

In [23]:
count_df.head()

,abandon,abc,abc news,abduct,abe,abedin,abl,abort,abroad,absolut,...,zero,zika,zika viru,zionist,zone,zone new,zone new york,zoo,zu,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [25]:
from sklearn.naive_bayes import MultinomialNB

In [26]:
classifier = MultinomialNB()

In [27]:
from sklearn import metrics
import numpy as np
import itertools

In [30]:
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test,y_pred)
print(score)
cm = metrics.confusion_matrix(y_test,y_pred)
print(cm)

0.9010770505385253
[[3048  341]
 [ 256 2390]]


In [31]:
#Passive Aggressive Classifier Algorithm.

In [35]:
from sklearn.linear_model import PassiveAggressiveClassifier
linear_clf = PassiveAggressiveClassifier()

In [36]:
linear_clf.fit(X_train,y_train)
y_pred = linear_clf.predict(X_test)
score = metrics.accuracy_score(y_test,y_pred)
print(score)
cm = metrics.confusion_matrix(y_test,y_pred)
print(cm)

0.9168185584092792
[[3147  242]
 [ 260 2386]]


In [37]:
#Multinomial classifier with HyperParameter

In [38]:
classifier = MultinomialNB(alpha=0.1)

In [39]:
previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier = MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    y_pred = sub_classifier.predict(X_test)
    score = metrics.accuracy_score(y_test,y_pred)
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha:{},Score:{}".format(alpha,score))

C:\Users\deepg\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha:0.0,Score:0.8886495443247722
Alpha:0.1,Score:0.9009113504556753
Alpha:0.2,Score:0.9005799502899752
Alpha:0.30000000000000004,Score:0.9002485501242751
Alpha:0.4,Score:0.8994200497100249
Alpha:0.5,Score:0.899917149958575
Alpha:0.6000000000000001,Score:0.9002485501242751
Alpha:0.7000000000000001,Score:0.9009113504556753
Alpha:0.8,Score:0.899751449875725
Alpha:0.9,Score:0.9004142502071251


In [40]:
feature_names = cv.get_feature_names()

In [41]:
classifier.coef_[0]

array([ -9.09822238,  -8.62059483,  -9.09822238, ..., -10.7928181 ,
        -8.91250524,  -9.32648103])

In [42]:
#Most real
sorted(zip(classifier.coef_[0],feature_names),reverse=True)[:20]

[(-3.997982703835758, 'trump'),
 (-4.289029055829791, 'hillari'),
 (-4.396073649945432, 'clinton'),
 (-4.897803273439508, 'elect'),
 (-5.1744321481285285, 'new'),
 (-5.23256391357954, 'comment'),
 (-5.271801728204403, 'video'),
 (-5.384650315348784, 'war'),
 (-5.399695734561153, 'us'),
 (-5.409853262219178, 'hillari clinton'),
 (-5.414970980656158, 'fbi'),
 (-5.478518031772853, 'vote'),
 (-5.5640890230861775, 'email'),
 (-5.576072389973274, 'world'),
 (-5.631848927430686, 'obama'),
 (-5.732335003003676, 'donald'),
 (-5.760928802370417, 'donald trump'),
 (-5.782923823955963, 'russia'),
 (-5.844058212449331, 'day'),
 (-5.859944170038141, 'america')]

In [43]:
#Most Fake
sorted(zip(classifier.coef_[0],feature_names))[:20]

[(-10.7928181028275, 'abe'),
 (-10.7928181028275, 'abroad'),
 (-10.7928181028275, 'abus new'),
 (-10.7928181028275, 'abus new york'),
 (-10.7928181028275, 'act new'),
 (-10.7928181028275, 'act new york'),
 (-10.7928181028275, 'advic'),
 (-10.7928181028275, 'advis new'),
 (-10.7928181028275, 'advis new york'),
 (-10.7928181028275, 'afford care'),
 (-10.7928181028275, 'afford care act'),
 (-10.7928181028275, 'age new'),
 (-10.7928181028275, 'age new york'),
 (-10.7928181028275, 'agenda breitbart'),
 (-10.7928181028275, 'ail'),
 (-10.7928181028275, 'aleppo new'),
 (-10.7928181028275, 'aleppo new york'),
 (-10.7928181028275, 'ali'),
 (-10.7928181028275, 'america breitbart'),
 (-10.7928181028275, 'america new york')]

In [44]:
#TFIDF